In [3]:
#This notebook will explore teams by play type. Including some clustering algorthim work

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
df = pd.read_csv('../data/interim/offensive_plays.csv', low_memory=False)

In [10]:
#Trim to just 2019 Data for now

In [3]:
df_2019 = df.loc[df['season']==2019]

In [99]:
#Takes a dataframe of cleaned plays and creates a new dataframe that grabs the number of pass and run plays by team. 
#Also finds the percent of total offensive plays (not including punts, fgs etc. for pass and run)

def make_team_play_db(df):
    
    df_ptypes = df[['posteam','play_type','play_id']] #grabs the required data from the large dataframe
    
    df_play_breakdown = df_ptypes.groupby(['posteam','play_type']).count() #Groups the dataframe by team and then play type and then counts the number of each type of paly
    
    teams = ['GB', 'CHI', 'CAR', 'LAR', 'WAS', 'PHI', 'BUF', 'NYJ', 'ATL',
           'MIN', 'MIA', 'BAL', 'KC', 'JAX', 'CLE', 'LAC', 'TEN', 'IND',
           'CIN', 'SEA', 'TB', 'SF', 'DAL', 'NYG', 'DET', 'ARI', 'NE', 'PIT',
           'NO', 'HOU', 'OAK', 'DEN']

    lst_teams = []
    lst_passes = []
    lst_runs = []
    lst_total = []
    lst_season = []

    for team in teams:
        lst_teams.append(team)
        lst_season.append(2019)
        lst_passes.append(df_play_breakdown['play_id'][team]['pass'])
        lst_runs.append(df_play_breakdown['play_id'][team]['run'])
        lst_total.append(df_play_breakdown['play_id'][team]['pass'] + df_play_breakdown['play_id'][team]['run'])

    df_team_plays = pd.DataFrame(zip(lst_teams, lst_season, lst_passes, lst_runs, lst_total),columns=['team','season','passes','runs','total']).sort_values(by='team').reset_index(drop=True)

    #Add the percentage of each type
    
    df_team_plays['pass_percent'] = df_team_plays['passes'] / df_team_plays['total'] * 100
    df_team_plays['run_percent'] = df_team_plays['runs'] / df_team_plays['total'] * 100

    #Add air yards
    
    df_air_yards = df[['posteam','air_yards']].groupby('posteam').sum().rename_axis('team').reset_index()
    df_team_plays = df_team_plays.join(df_air_yards,rsuffix='test').drop('teamtest',axis=1)
    df_team_plays['air_yards_per_att'] = df_team_plays['air_yards'] / df_team_plays['passes']
    
    #Runs
    df_runs = df[['posteam','run_gap','play_id','play_type','fumble']]
    df_runs = df_runs.loc[(df_runs['play_type']=='run') & (df_runs['fumble']==0)] #only runs and non fumbles
    df_runs['run_gap'].fillna('middle',axis=0,inplace=True)
    df_runs.drop(['play_type','fumble','play_id'],axis=1,inplace=True)
    
    #Tackle Gap
    df_run_tackle = df_runs.loc[df_runs['run_gap']=='tackle'].groupby('posteam').count().rename({'run_gap':'run_gap_tackle'},axis=1).reset_index()
    df_team_plays = pd.concat([df_team_plays, df_run_tackle['run_gap_tackle']],axis=1)  
    
    #End Gap
    df_run_tackle = df_runs.loc[df_runs['run_gap']=='end'].groupby('posteam').count().rename({'run_gap':'run_gap_end'},axis=1).reset_index()
    df_team_plays = pd.concat([df_team_plays, df_run_tackle['run_gap_end']],axis=1)  
    
    #Guard Gap
    df_run_tackle = df_runs.loc[df_runs['run_gap']=='guard'].groupby('posteam').count().rename({'run_gap':'run_gap_guard'},axis=1).reset_index()
    df_team_plays = pd.concat([df_team_plays, df_run_tackle['run_gap_guard']],axis=1)  
    
    #Middle
    df_run_tackle = df_runs.loc[df_runs['run_gap']=='middle'].groupby('posteam').count().rename({'run_gap':'run_gap_middle'},axis=1).reset_index()
    df_team_plays = pd.concat([df_team_plays, df_run_tackle['run_gap_middle']],axis=1)  
    
    #Percentage for each
    
    df_team_plays['run_gap_tackle_percent'] = df_team_plays['run_gap_tackle'] / df_team_plays['runs'] *100
    
    df_team_plays['run_gap_end_percent'] = df_team_plays['run_gap_end'] / df_team_plays['runs'] *100
    
    df_team_plays['run_gap_guard_percent'] = df_team_plays['run_gap_guard'] / df_team_plays['runs'] *100
    
    df_team_plays['run_gap_middle_percent'] = df_team_plays['run_gap_middle'] / df_team_plays['runs'] *100
    
    
    return df_team_plays


In [100]:
df_team_plays = make_team_play_db(df_2019)

In [101]:
df_team_plays

,team,season,passes,runs,total,pass_percent,run_percent,air_yards,air_yards_per_att,run_gap_tackle,run_gap_end,run_gap_guard,run_gap_middle,run_gap_tackle_percent,run_gap_end_percent,run_gap_guard_percent,run_gap_middle_percent
0,ARI,2019,597,385,982,60.794297,39.205703,4086.0,6.844221,42,152,41,147,10.909091,39.480519,10.649351,38.181818
1,ATL,2019,729,350,1079,67.562558,32.437442,5485.0,7.524005,101,85,105,55,28.857143,24.285714,30.000000,15.714286
2,BAL,2019,466,571,1037,44.937319,55.062681,3822.0,8.201717,152,96,206,103,26.619965,16.812609,36.077058,18.038529
3,BUF,2019,545,452,997,54.663992,45.336008,4679.0,8.585321,105,62,134,138,23.230088,13.716814,29.646018,30.530973
4,CAR,2019,687,381,1068,64.325843,35.674157,5241.0,7.628821,82,87,97,108,21.522310,22.834646,25.459318,28.346457
5,CHI,2019,622,389,1011,61.523244,38.476756,4521.0,7.268489,98,97,124,65,25.192802,24.935733,31.876607,16.709512
6,CIN,2019,652,377,1029,63.362488,36.637512,4526.0,6.941718,78,80,135,80,20.689655,21.220159,35.809019,21.220159
7,CLE,2019,576,387,963,59.813084,40.186916,4546.0,7.892361,101,51,157,73,26.098191,13.178295,40.568475,18.863049
8,DAL,2019,617,439,1056,58.428030,41.571970,5510.0,8.930308,122,108,113,89,27.790433,24.601367,25.740319,20.273349
9,DEN,2019,542,398,940,57.659574,42.340426,3451.0,6.367159,91,99,70,135,22.864322,24.874372,17.587940,33.919598
